In [20]:

import wandb
import warnings
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from dice_ml.explainer_interfaces.explainer_base import UserConfigValidationException
import plotly.graph_objects as go
from datetime import timedelta
import os
import re
import joblib
from tslearn.neighbors import KNeighborsTimeSeries

min_max_scaler = MinMaxScaler()
warnings.filterwarnings("ignore")
abt_path='../../'


In [ ]:
# wandb.init(
#     entity="gsu-dmlab",
#     # project="SEPCFE",  # You choose the project name
#     name="NG_exp1_Onboarding",  # Give each run a name
# )

In [6]:
abt_path_train = abt_path+'data/raw/mean_sliced/train_data.csv'
df_train = pd.read_csv(abt_path_train,  sep=',',index_col=0)
abt_path_test = abt_path+'data/raw/mean_sliced/test_data.csv'
df_test = pd.read_csv(abt_path_test, sep=',',index_col=0)
abt_path_val = abt_path+'data/raw/mean_sliced/valid_data.csv'
df_val = pd.read_csv(abt_path_val, sep=',',index_col=0)

In [10]:
X_train=pd.read_csv(abt_path+'data/processed/X_train.csv',sep=',',index_col=0)
Y_train=pd.read_csv(abt_path+'data/processed/Y_train.csv',sep=',',index_col=0)
X_test=pd.read_csv(abt_path+'data/processed/X_test.csv',sep=',',index_col=0)
Y_test=pd.read_csv(abt_path+'data/processed/Y_test.csv',sep=',',index_col=0)
X_val=pd.read_csv(abt_path+'data/processed/X_val.csv',sep=',',index_col=0)
Y_val=pd.read_csv(abt_path+'data/processed/Y_val.csv',sep=',',index_col=0)
df_multi_labels = pd.read_excel(abt_path+"data/raw/multi_labels.xlsx",header=0)
df_multi_labels = df_multi_labels[df_multi_labels['File'] != '1991-06-18_23-16.csv']
df_combined_labels=pd.read_csv(abt_path+'data/processed/df_combined_labels.csv',sep=',',index_col=0)

In [13]:
p3_flux_cols = [col for col in df_train.columns if 'p3_flux_ic_mean' in col]
X_train_p3 = X_train[p3_flux_cols].copy()

In [19]:
importances = best_rf.feature_importances_

In [31]:
def native_guide_retrieval(query, predicted_label, distance, n_neighbors):

    df = pd.DataFrame(Y_train, columns = ['label'])
    df.index.name = 'index'
    df[df['label'] == 1].index.values, df[df['label'] != 1].index.values

    ts_length = X_train.shape[1]


    knn = KNeighborsTimeSeries(n_neighbors=n_neighbors, metric = distance)

    knn.fit(X_train.iloc[df[df['label'] != predicted_label].index.values])

    dist, ind = knn.kneighbors(query.values.reshape(1, ts_length), return_distance=True)
    return dist[0], df[df['label'] != predicted_label].index[ind[0][:]]

In [32]:
# Load model
best_rf = joblib.load(abt_path+'/models/RandomForestClassifier_model_sep_cfe.pkl')
Y_pred = best_rf.predict(X_test)

nuns = []
for instance in range(len(X_test)):
    x_row = X_test.iloc[instance]  # Correct way to get row `instance`
    y_pred = Y_pred[instance]
    print(instance)

    nuns.append(native_guide_retrieval(x_row,y_pred, 'euclidean', 1)[1][0])
nuns = np.array(nuns)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [33]:
nuns

array([ 23,  61,  26,  27,  27, 718,  63,  29,  52, 435,  29,  30,  29,
        85,  30, 214,  29, 132, 132,  68,  64,  28, 177,  50,  29,  29,
        29,  29,  29,  29,  68, 128, 128,  64, 810, 393, 513, 386, 178,
       178,  98, 244,  82, 283, 128,  68,  95,  49,  96, 630, 289,  96,
       134,  30,  68,  29, 155,  29,  30, 400,  63, 145, 100,  27,  30,
        29,  29,  29,  30, 257,  30,  29, 343, 338, 698, 810, 752, 752,
       513, 743,  28,  68,  75, 669, 722, 515, 283,  63,  63,  24,  68,
       170,  80,  64,  29,  32, 101,  49,  41, 156, 587, 289,  29,  52,
         1,  75,  29,  27,  29,  64, 170,  27,  52,  85,  38,  29,  29,
        93,  29,  47,  52,  92,  29,  35,  62,  39,  32,  30,  29,  29,
        47,  28,  32,  27,  30,  52,  92,  29, 732, 189,  25,  28,  29,
        93,  61, 703, 393, 693, 175, 977,   5, 507, 357,  16,  31,  39,
       947,   2,  52,  29,  12, 667, 158, 517,  45,  27,  37,  40,  46,
        31,  50, 919, 791,  24, 342, 858,  50,  29,  28,  36, 51

In [83]:
import shap
instance_index = 0
instance = X_test.iloc[[instance_index]]
# Use correct SHAP explainer
explainer = shap.Explainer(best_rf)
instance_shap_values=[]
for i in range(len(X_test)):
    instance = X_test.iloc[[i]]
    shap_values = explainer.shap_values(instance)
    instance_shap_values.append(shap_values)
instance_shap_values=np.array(instance_shap_values)

In [84]:
def findSubarray(a, k): #used to find the maximum contigious subarray of length k in the explanation weight vector

    n = len(a)

    vec=[]

    # Iterate to find all the sub-arrays
    for i in range(n-k+1):
        temp=[]

        # Store the sub-array elements in the array
        for j in range(i,i+k):
            temp.append(a[j])

        # Push the vector in the container
        vec.append(temp)

    sum_arr = []
    for v in vec:
        sum_arr.append(np.sum(v))

    return (vec[np.argmax(sum_arr)])

In [98]:
def counterfactual_generator_swap(instance, nun, subarray_length):

    most_influencial_array=findSubarray((instance_shap_values[nun]), subarray_length)

    starting_point = np.where(instance_shap_values[nun]==most_influencial_array[0])[0][0]

    X_example = np.concatenate((X_test.iloc[instance][:starting_point], (X_train.iloc[nun][starting_point:subarray_length+starting_point]), X_test.iloc[instance][subarray_length+starting_point:]))

    prob_target = best_rf.predict(X_example.reshape(-1,928))[0][y_pred[instance]]

    while prob_target > 0.5:

        subarray_length +=1

        most_influencial_array=findSubarray((instance_shap_values[nun]), subarray_length)
        starting_point = np.where(instance_shap_values[nun]==most_influencial_array[0])[0][0]
        X_example = np.concatenate((X_test.iloc[instance][:starting_point], (X_train.iloc[nun][starting_point:subarray_length+starting_point]), X_test.iloc[instance][subarray_length+starting_point:]))
        prob_target = best_rf.predict(X_example.reshape(-1,928))[0][y_pred[instance]]

    return X_example

In [99]:
test_instances = np.array(range(len(X_test)))
test_instances

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [100]:
cf_cam_swap = []

for test_instance, nun in zip(test_instances, nuns):
    cf_cam_swap.append(counterfactual_generator_swap(test_instance,nun,1))

IndexError: invalid index to scalar variable.

In [94]:
y_pred

0